In [3]:
import numpy as np
import pandas as pd

from keras.models import Model
from keras.layers import Conv2D, Input, Flatten, Dense, Reshape,UpSampling2D,ZeroPadding2D, MaxPooling2D

In [2]:
X = pd.read_csv("/home/vasu/Downloads/fashion-mnist_train.csv").values

y = X[:,0]
X = X[:,1:]
X = X/255.0

X = X.reshape((X.shape[0],28,28,1))

print (X.shape, y.shape)

(60000, 28, 28, 1) (60000,)


In [5]:
## Encoder

inp = Input(shape=(28,28,1))
c1 = Conv2D(32, (3,3),activation='relu')(inp)
m1 = MaxPooling2D((2,2))(c1)
c2 = Conv2D(64, (3,3), activation='relu')(m1)
m2 = MaxPooling2D((2,2))(c2)
c3 = Conv2D(8, (3,3), activation='relu')(m2)
f1 = Flatten()(c3)
d1 = Dense(2, activation='relu')(f1)

## Decoder

d2 = Dense(72, activation='relu')(d1)
r1 = Reshape((3,3,8))(d2)
zp1 = ZeroPadding2D((1,1))(r1)
c4 = Conv2D(64, (3,3), activation='relu', padding='same')(zp1)
up1 = UpSampling2D((2,2))(c4)
zp2 = ZeroPadding2D((2,2))(up1)
c5 = Conv2D(32, (3,3), activation='relu', padding='same')(zp2)
up2 = UpSampling2D((2,2))(c5)
c6 = Conv2D(1, (3,3), activation='sigmoid', padding='same')(up2)

model = Model(inputs=inp, outputs=c6)
encoder = Model(inputs=inp, outputs=d1)

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3, 3, 8)           4616      
_________________________________________________________________
flatten_2 (Flatten)          (None, 72)                0         
__________

In [7]:
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3, 3, 8)           4616      
_________________________________________________________________
flatten_2 (Flatten)          (None, 72)                0         
__________

In [8]:
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [9]:
hist = model.fit(X,X, epochs=20, shuffle=True, batch_size=128, validation_split=0.20)

Instructions for updating:
Use tf.cast instead.
Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 127s 3ms/step - loss: 0.0598 - acc: 0.4940 - val_loss: 0.0409 - val_acc: 0.4956
Epoch 2/20
48000/48000 [==============================] - 135s 3ms/step - loss: 0.0392 - acc: 0.4958 - val_loss: 0.0383 - val_acc: 0.4966
Epoch 3/20
48000/48000 [==============================] - 135s 3ms/step - loss: 0.0375 - acc: 0.4964 - val_loss: 0.0373 - val_acc: 0.4978
Epoch 4/20
48000/48000 [==============================] - 137s 3ms/step - loss: 0.0365 - acc: 0.4970 - val_loss: 0.0369 - val_acc: 0.4960
Epoch 5/20
48000/48000 [==============================] - 137s 3ms/step - loss: 0.0358 - acc: 0.4973 - val_loss: 0.0361 - val_acc: 0.4943
Epoch 6/20
48000/48000 [==============================] - 142s 3ms/step - loss: 0.0353 - acc: 0.4977 - val_loss: 0.0352 - val_acc: 0.4968
Epoch 7/20
48000/48000 [==============================] - 138s 3ms/step - 

In [10]:
encoder.save("encoder.h5")

In [11]:
model.save("autoencoder.h5")